# Jupyter Notebook Testing

Jupyter Notebooks use cells that can easily be re ran and is very convenient for testing things out.
It's important to note that to be able to run these cells correctly, the dependencies need to be properly installed.
This means that the interpreter being used must point to the Python the one that is in your [venv](./.venv) created by poetry for this project.

**Note**: if you make changes to the source code, you will need to press `Restart` above to restart the kernel to ensure the changes occur here.
![image](../pictures/restartJupyterKernel.png)

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
from backend.engine.types.IndustryType import IndustryType


tax_rates = {
    "corporate_income_tax": {itype.value: 0.0 for itype in IndustryType},
    "personal_income_tax": 0.0,
    "sales_tax": {itype.value: 0.0 for itype in IndustryType},
    "property_tax": 0.0,
    "tariffs": {itype.value: 0.0 for itype in IndustryType},
}

In [ ]:
from backend.engine.interface.controller import ModelController
controller = ModelController()
model_id = "model1"
controller.create_model(model_id=model_id, city_template=None, num_people=100,tax_rates=tax_rates)

In [3]:
controller.step_model(model_id)

INFO:MESA.mesa.model:calling model.step for timestep 1 
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Producing goods...NOT IMPLEMENTED
INFO:root:Producing goods...NOT IMPLEMENTED
INFO:root:Producing goods...NOT IMPLEMENTED
INFO:root:Producing goods...NOT IMPLEMENTED
INFO:root:Producin

In [33]:
model = controller.get_model(model_id)

indicators = model.datacollector.get_model_vars_dataframe()

print(indicators)

   Week  Unemployment  GDP  IncomePerCapita  MedianIncome  HooverIndex  \
0     0           1.0    0         50.60199     50.119398            0   
1     1           1.0    0         51.60199     51.119398            0   

   LorenzCurve  
0            0  
1            0  
